In [1]:
import os
import tflearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline
%tensorflow_version 1.x

In [2]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from string import punctuation
import re

Using TensorFlow backend.


In [9]:
df = pd.read_csv('/content/labeledTrainData.tsv',sep='\t')
df.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [10]:
df.describe()

,sentiment
count,25000.00000
mean,0.50000
std,0.50001
min,0.00000
25%,0.00000
50%,0.50000
75%,1.00000
max,1.00000


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
id           25000 non-null object
sentiment    25000 non-null int64
review       25000 non-null object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


In [12]:
df.sentiment.value_counts()

1    12500
0    12500
Name: sentiment, dtype: int64

In [0]:
def cleaned_data(input_sentence):
  
  input_sentence = input_sentence.lower()
  input_sentence = re.sub(r"<br />", " ", input_sentence)
  input_sentence = re.sub(r"   ", " ", input_sentence) # Remove any extra spaces
  input_sentence = re.sub(r"  ", " ", input_sentence)
  text = ''.join([c for c in input_sentence if c not in punctuation])
  cleaned_data =  re.compile("[^a-z ]+")
  return re.sub(cleaned_data," ",input_sentence)

In [14]:
df['review'] = df['review'].apply(cleaned_data)
df.head()

,id,sentiment,review
0,5814_8,1,with all this stuff going down at the moment w...
1,2381_9,1,the classic war of the worlds by timothy hin...
2,7759_3,0,the film starts with a manager nicholas bell ...
3,3630_4,0,it must be assumed that those who praised this...
4,9495_8,1,superbly trashy and wondrously unpretentious ...


In [15]:
lengths = df['review'].apply(lambda x : len(x.split()) )
lengths_df = pd.DataFrame(lengths)
lengths_df.head()

,review
0,437
1,161
2,375
3,384
4,379


In [16]:
lengths.describe()

count    25000.000000
mean       236.871680
std        176.011452
min         10.000000
25%        129.000000
50%        177.000000
75%        287.000000
max       2494.000000
Name: review, dtype: float64

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re


tokenizer = Tokenizer(split=' ')
tokenizer.fit_on_texts(df['review'].values)
X = tokenizer.texts_to_sequences(df['review'].values)
X = pad_sequences(X,400)


In [20]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(9000, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())


Y = pd.get_dummies(df['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)




Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 400, 128)          1152000   
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 400, 128)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 394       
Total params: 1,407,194
Trainable params: 1,407,194
Non-trainable params: 0
_________________________________________________________________
None
(16750, 400) (16750, 2)
(8250, 400) (8250, 2)


In [21]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)



Epoch 1/7
 - 459s - loss: 0.5512 - acc: 0.7149
Epoch 2/7
 - 456s - loss: 0.4180 - acc: 0.8214
Epoch 3/7
 - 459s - loss: 0.3358 - acc: 0.8613
Epoch 4/7
 - 462s - loss: 0.2941 - acc: 0.8815
Epoch 5/7
 - 461s - loss: 0.2516 - acc: 0.9044
Epoch 6/7
 - 462s - loss: 0.1966 - acc: 0.9281
Epoch 7/7
 - 459s - loss: 0.1530 - acc: 0.9418


In [0]:
model.save('94_sentiment.h5')

In [23]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))



score: 0.46
acc: 0.83


In [24]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

pos_acc 78.00269905533064 %
neg_acc 90.9090909090909 %


In [25]:
import tflearn
from tflearn.data_utils import to_categorical, pad_sequences
from tflearn.datasets import imdb
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.embedding_ops import embedding
from tflearn.layers.recurrent import bidirectional_rnn, BasicLSTMCell
from tflearn.layers.estimator import regression

net = input_data(shape=[None, 400])
net = embedding(net, input_dim=20000, output_dim=128)
net = bidirectional_rnn(net, BasicLSTMCell(128), BasicLSTMCell(128))
net = dropout(net, 0.5)
net = fully_connected(net, 2, activation='softmax')
net = regression(net, optimizer='adam', loss='categorical_crossentropy')

# Training
model = tflearn.DNN(net, clip_gradients=0., tensorboard_verbose=2)
model.fit(X_train, y_train, validation_set=0.1, show_metric=True, batch_size=64)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


ValueError: ignored

In [0]:
test_df  = pd.read_csv('/content/testData.tsv',sep='\t')
test_df.head()

,id,review
0,12311_10,Naturally in a film who's main themes are of m...
1,8348_2,This movie is a disaster within a disaster fil...
2,5828_4,"All in all, this is a movie for kids. We saw i..."
3,7186_2,Afraid of the Dark left me with the impression...
4,12128_7,A very accurate depiction of small time mob li...


In [0]:
test_df['review'] = test_df['review'].apply(cleaned_data)
test_df.head()

,id,review
0,12311_10,naturally in a film who s main themes are of m...
1,8348_2,this movie is a disaster within a disaster fil...
2,5828_4,all in all this is a movie for kids we saw i...
3,7186_2,afraid of the dark left me with the impression...
4,12128_7,a very accurate depiction of small time mob li...
